In [1]:
import pandas as pd
import numpy as np
import torch
import optuna
import torchviz
import torch.nn as nn
from torchsummary import summary
from torch.nn.functional import mse_loss
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from utils_pp import replace_cell_names_with_id
%load_ext autoreload
%autoreload 2

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

c:\Users\antoi\ai-imbsl\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# columns = ["cell_line", "drugA_name", "drugB_name", "drugA_conc", "drugB_conc", "target"]
# df_train = pd.read_csv("../data_raw/oneil.csv", usecols=(1,2,3,4,5,12)).iloc[:,[0,1,3,2,4,5]].set_axis(columns, axis=1)
# df_train["cell_line"]
# df_test = pd.read_csv("../data/test_yosua.csv").set_axis(columns + ["std"], axis=1).convert_dtypes()

drug_data = pd.read_pickle("../data/drug_data.pkl.compress", compression="gzip")
cell_data = pd.read_pickle("../data/cell_line_data.pkl.compress", compression="gzip")
drug_data_val = drug_data.copy()
X_train_drug, X_val_drug, y_train_drug, y_val_drug = train_test_split(drug_data, drug_data_val, test_size=0.2, shuffle=True)
# df_train = replace_cell_names_with_id(dataframe=df_train, mapping_file="../data/mappingccl.csv")
# df_test = replace_cell_names_with_id(dataframe=df_test, mapping_file="../data/mappingccl.csv")
# df_train = df_train[df_train.cell_line.isin(cell_data.index)]
# df_train, df_val = train_test_split(df_train, test_size=0.2, shuffle=True)
# cell_data = cell_data[cell_data.index.isin(pd.concat([df_train.cell_line, df_test.cell_line]))]
# print("oneil", df_train.memory_usage().sum()/1e6, df_train.shape,"\n", df_train.dtypes)
print("drug_feat", X_train_drug.memory_usage().sum()/1e6, X_train_drug.shape)
print("drug_feat", X_val_drug.memory_usage().sum()/1e6, X_val_drug.shape)
print("cell_feat", cell_data.memory_usage().sum()/1e6, cell_data.shape)


drug_feat 5.213636 (538, 2412)
drug_feat 1.324686 (135, 2412)
cell_feat 29.111364 (1450, 5011)


In [3]:
class dataset(Dataset):
    def __init__(self, data, labels):
        self.dataset = data.to_numpy()
        self.labels = labels.to_numpy()

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx], self.labels[idx]
                
train_set_drug  = dataset(X_train_drug, y_train_drug)
val_set_drug  = dataset(X_val_drug, y_val_drug)
train_dl_drug = DataLoader(train_set_drug, batch_size=64, shuffle=True)
val_dl_drug = DataLoader(val_set_drug, batch_size=64, shuffle=True)
xi, yi = next(iter(train_dl_drug))
print(xi.shape, yi.shape)

torch.Size([64, 2412]) torch.Size([64, 2412])


In [4]:

# Creating a PyTorch class
# 28*28 ==> 9 ==> 28*28
class AE(torch.nn.Module):
    def __init__(self, h_sizes, dropout=0.2):
        super().__init__()
         
        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 ==> 9
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(h_sizes[0], h_sizes[1]),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(h_sizes[1], h_sizes[2]),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(h_sizes[2], h_sizes[3])
        )
         
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 9 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(h_sizes[3], h_sizes[4]),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(h_sizes[4], h_sizes[5]),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(h_sizes[5], h_sizes[0])
        )
 
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
ae = AE([xi.shape[1],512,512,256,512,512])
yi = ae.forward(xi)
print(xi.shape, yi.shape)

torch.Size([64, 2412]) torch.Size([64, 2412])


In [10]:
from torchview import draw_graph

# device='meta' -> no memory is consumed for visualization
model_graph = draw_graph(ae, input_size=(64, 2412), device='meta', graph_name='MLP autoencoder', graph_dir='LR')
model_graph.visual_graph


(dot.exe:14968): Pango-WARNING **: couldn't load font "Linux libertine Not-Rotated 10", falling back to "Sans Not-Rotated 10", expect ugly output.


In [5]:
def train_one_epoch(model, epoch_index, tb_writer, training_loader, optimizer, loss_fn, device, L1, verbose=False):
    running_loss = 0.
    last_loss = 0.
    model = model.to(device)
    for i, data in enumerate(training_loader):
        inputs, labels = data
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        outputs = model.forward(inputs)

        params = torch.cat([x.view(-1) for x in model.parameters()])
        l1_regularization = L1 * torch.linalg.vector_norm(params, 1)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels) + l1_regularization
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        running_loss += loss.item()

        # last_loss =  loss.item() / inputs.shape[0]  # loss per sample
        # if verbose:print('  batch {} loss: {}'.format(i + 1, last_loss), outputs[0][0].item(), labels[0][0].item())
        # tb_x = epoch_index * len(training_loader) + i + 1
        # tb_writer.add_scalar('Loss/train', last_loss, tb_x)
    
    return running_loss / (i+1)

In [6]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [7]:
import optuna
batch_size=256
train_set_drug  = dataset(X_train_drug, y_train_drug)
val_set_drug  = dataset(X_val_drug, y_val_drug)
train_dl_drug = DataLoader(train_set_drug, batch_size=batch_size, shuffle=True)
val_dl_drug = DataLoader(val_set_drug, batch_size=batch_size, shuffle=True)
xi, yi = next(iter(train_dl_drug))

def objective(trial=None):
    if trial is None:
        lr = 1e-3
        dropout=0.2
        weight_decay=1e-4
        L1 = 1e-6
        first_layer=1024
    else:
        weight_decay = trial.suggest_float("weight_decay", 1e-8, 1e-4, log=True)
        L1 = trial.suggest_float("L1", 1e-8, 1e-4, log=True)

        lr = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
        dropout = trial.suggest_float("dropout", 0.0, 0.3) 
        first_layer = trial.suggest_categorical("first_layer", [1024, 512,256,128])

    ae = AE([xi.shape[1],first_layer,512,256,512,first_layer], dropout=dropout)
    # print(summary(ae.to("cuda"), xi.shape))
    early_stopper = EarlyStopper(patience=20)
    optimizer = torch.optim.Adam(ae.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=0.5, verbose=True, patience=10, min_lr=1e-7)
    loss_fn = torch.nn.MSELoss()
    # Initializing in a separate cell so we can easily add more epochs to the same run
    # timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    # writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
    epoch_number = 0

    EPOCHS = 500

    for epoch in range(EPOCHS):
        # Make sure gradient tracking is on, and do a pass over the data
        ae.train(True)
        ae = ae.to(device=device)

        avg_loss = train_one_epoch(ae, epoch_number, "writer", train_dl_drug, optimizer, loss_fn, L1=L1,device=device)

        running_vloss = 0.0
        # Set the ae to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        ae.eval()

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i, vdata in enumerate(val_dl_drug):
                vinputs, vlabels = vdata
                vinputs = vinputs.to(device)
                vlabels = vlabels.to(device)
                voutputs = ae(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                running_vloss += vloss
        avg_vloss = running_vloss / (i + 1) 
        print(vinputs.shape[0])

        if epoch_number%10==9:print('EPOCH {}:'.format(epoch_number + 1),'LOSS train {} valid {}'.format(avg_loss, avg_vloss))
        scheduler.step(avg_loss)

        if early_stopper.early_stop(avg_vloss):             
            break

        if trial is not None:
            trial.report(avg_vloss, epoch_number)
            if trial.should_prune():
                raise optuna.TrialPruned()
        epoch_number += 1

    return avg_vloss

# objective()

# with torch.no_grad():
#     for i, data in enumerate(train_dl_drug):
#         inputs, labels = data
#         inputs = inputs.to(device=device)
#         labels = labels.to(device=device)
#         outputs = ae(inputs)
# with open("output.txt", mode="w") as f: 
#     [f.write(
#         str(outputs[0][i].to("cpu").numpy().round(3))+"   "+
#         str(inputs[0][i].to("cpu").numpy().round(3))+"\n") for i in range(len(outputs[0]))]

In [9]:
batch_size=256
train_set_drug  = dataset(X_train_drug, y_train_drug)
val_set_drug  = dataset(X_val_drug, y_val_drug)
train_dl_drug = DataLoader(train_set_drug, batch_size=batch_size, shuffle=True)
val_dl_drug = DataLoader(val_set_drug, batch_size=1, shuffle=True)
xi, yi = next(iter(train_dl_drug))

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=100)

[I 2023-08-24 17:55:47,345] A new study created in memory with name: no-name-2300212f-d3ef-4223-ad49-b08e5bfafde9


EPOCH 10: LOSS train 0.9558379650115967 valid 0.9500653147697449
EPOCH 20: LOSS train 0.9222928285598755 valid 0.9241638779640198
EPOCH 30: LOSS train 0.8798766930898031 valid 0.9081265330314636
EPOCH 40: LOSS train 0.8756439487139384 valid 0.8905343413352966
Epoch 00040: reducing learning rate of group 0 to 4.6252e-04.
EPOCH 50: LOSS train 0.8701503674189249 valid 0.8774283528327942
EPOCH 60: LOSS train 0.8431068460146586 valid 0.8735763430595398
EPOCH 70: LOSS train 0.842602550983429 valid 0.8666717410087585
EPOCH 80: LOSS train 0.8455469806989034 valid 0.8602144718170166
Epoch 00089: reducing learning rate of group 0 to 2.3126e-04.
EPOCH 90: LOSS train 0.8269160985946655 valid 0.8573508262634277
EPOCH 100: LOSS train 0.8092964887619019 valid 0.852138876914978
EPOCH 110: LOSS train 0.8178675174713135 valid 0.8505203723907471
Epoch 00115: reducing learning rate of group 0 to 1.1563e-04.
EPOCH 120: LOSS train 0.7934661308924357 valid 0.8491114974021912
EPOCH 130: LOSS train 0.805749515

[I 2023-08-24 17:56:25,683] Trial 0 finished with value: 0.8453511595726013 and parameters: {'weight_decay': 1.521503859743021e-07, 'L1': 3.0320881973172606e-08, 'learning_rate': 0.000925047971446074, 'dropout': 0.22367883161577556, 'first_layer': 128}. Best is trial 0 with value: 0.8453511595726013.


EPOCH 10: LOSS train 5.749079863230388 valid 0.9661383628845215
EPOCH 20: LOSS train 4.242317358652751 valid 0.9660544991493225
EPOCH 30: LOSS train 3.0814040501912436 valid 0.9660072326660156
EPOCH 40: LOSS train 2.2409890492757163 valid 0.9659820199012756
EPOCH 50: LOSS train 1.7390731970469158 valid 0.965957522392273
EPOCH 60: LOSS train 1.3637458483378093 valid 0.9659403562545776
EPOCH 70: LOSS train 1.1490124066670735 valid 0.96592777967453
EPOCH 80: LOSS train 1.1213864882787068 valid 0.9659102559089661
EPOCH 90: LOSS train 1.015136996905009 valid 0.9658880233764648
EPOCH 100: LOSS train 0.9945140679677328 valid 0.9658615589141846
EPOCH 110: LOSS train 0.9926746090253195 valid 0.965840756893158
Epoch 00119: reducing learning rate of group 0 to 7.3863e-05.
EPOCH 120: LOSS train 0.9918403029441833 valid 0.9658300280570984
EPOCH 130: LOSS train 1.0183249513308208 valid 0.9658229947090149
Epoch 00130: reducing learning rate of group 0 to 3.6932e-05.
EPOCH 140: LOSS train 1.0059877038

[I 2023-08-24 17:57:17,551] Trial 1 finished with value: 0.9658156633377075 and parameters: {'weight_decay': 9.397287637605942e-07, 'L1': 7.178700058262388e-05, 'learning_rate': 0.00014772615898265935, 'dropout': 0.05251964917304939, 'first_layer': 1024}. Best is trial 0 with value: 0.8453511595726013.


...

[I 2023-08-24 18:44:08,907] Trial 84 finished with value: 0.7099388837814331 and parameters: {'weight_decay': 7.92019577790693e-06, 'L1': 1.696170709020603e-07, 'learning_rate': 0.0007841433738117765, 'dropout': 0.04435050647117976, 'first_layer': 512}. Best is trial 84 with value: 0.7099388837814331.
[I 2023-08-24 18:44:09,398] Trial 85 pruned. 
[I 2023-08-24 18:44:09,888] Trial 86 pruned. 
[I 2023-08-24 18:44:10,308] Trial 87 pruned. 
[I 2023-08-24 18:44:10,712] Trial 88 pruned. 
[I 2023-08-24 18:44:11,125] Trial 89 pruned. 
[I 2023-08-24 18:44:11,515] Trial 90 pruned. 
[I 2023-08-24 18:44:15,058] Trial 91 pruned. 
[I 2023-08-24 18:44:15,893] Trial 92 pruned. 
[I 2023-08-24 18:44:16,403] Trial 93 pruned. 


EPOCH 10: LOSS train 0.899686594804128 valid 0.8953154683113098
EPOCH 20: LOSS train 0.8183322548866272 valid 0.8469147086143494
EPOCH 30: LOSS train 0.7127149105072021 valid 0.8194067478179932
EPOCH 40: LOSS train 0.6619896292686462 valid 0.7913077473640442
EPOCH 50: LOSS train 0.5936486124992371 valid 0.7840250134468079
EPOCH 60: LOSS train 0.511844664812088 valid 0.7699636816978455
EPOCH 70: LOSS train 0.47120201587677 valid 0.7545944452285767
EPOCH 80: LOSS train 0.4247824350992839 valid 0.7477700114250183
EPOCH 90: LOSS train 0.39837421973546344 valid 0.7475164532661438
EPOCH 100: LOSS train 0.36102710167566937 valid 0.7442284226417542
EPOCH 110: LOSS train 0.3677847683429718 valid 0.7421154379844666
EPOCH 120: LOSS train 0.34151419003804523 valid 0.738457202911377
EPOCH 130: LOSS train 0.3140309949715932 valid 0.7369002103805542
EPOCH 140: LOSS train 0.31626012921333313 valid 0.7380295395851135
EPOCH 150: LOSS train 0.2920709550380707 valid 0.7317113876342773
Epoch 00155: reducin

[I 2023-08-24 18:45:32,693] Trial 94 pruned. 
[I 2023-08-24 18:45:33,161] Trial 95 pruned. 


EPOCH 10: LOSS train 0.9015002051989237 valid 0.8991154432296753
EPOCH 20: LOSS train 0.7830127278963724 valid 0.8367441892623901
EPOCH 30: LOSS train 0.6777301033337911 valid 0.804577648639679
EPOCH 40: LOSS train 0.5520719091097513 valid 0.7840337157249451
EPOCH 50: LOSS train 0.4679309328397115 valid 0.7650690674781799
EPOCH 60: LOSS train 0.42824816703796387 valid 0.7604491710662842
EPOCH 70: LOSS train 0.3496352930863698 valid 0.7523722052574158
EPOCH 80: LOSS train 0.3041362265745799 valid 0.744291365146637
EPOCH 90: LOSS train 0.2670813699563344 valid 0.7400479912757874
EPOCH 100: LOSS train 0.2536432445049286 valid 0.7438588738441467
EPOCH 110: LOSS train 0.23398586610953012 valid 0.7390185594558716
EPOCH 120: LOSS train 0.20583285888036093 valid 0.735120952129364
EPOCH 130: LOSS train 0.20126953721046448 valid 0.7365003228187561
EPOCH 140: LOSS train 0.1972067952156067 valid 0.7308589220046997
EPOCH 150: LOSS train 0.19045096139113107 valid 0.7340288758277893
EPOCH 160: LOSS t

[I 2023-08-24 18:46:43,004] Trial 96 finished with value: 0.7411100268363953 and parameters: {'weight_decay': 1.3826991462510193e-05, 'L1': 6.521018687460175e-07, 'learning_rate': 0.0008790585193331017, 'dropout': 0.05262563814217478, 'first_layer': 1024}. Best is trial 84 with value: 0.7099388837814331.
[I 2023-08-24 18:46:43,556] Trial 97 pruned. 
[I 2023-08-24 18:46:44,075] Trial 98 pruned. 
[I 2023-08-24 18:46:44,558] Trial 99 pruned. 


In [8]:
param_dict_34 = {'weight_decay': 1.5949621641126727e-05, 'L1': 2.0214217845240095e-07, 'learning_rate': 0.0007584607894578174, 'dropout': 0.05293127067160355, 'first_layer': 512}
param_dict_46 = {'weight_decay': 1.4940271869850511e-05, 'L1': 2.035023742123312e-07, 'learning_rate': 0.0008215476035324593, 'dropout': 0.038985292466368573, 'first_layer': 512}
# 84
param_dict = {'weight_decay': 7.92019577790693e-06, 'L1': 1.696170709020603e-07, 'learning_rate': 0.0007841433738117765, 'dropout': 0.04435050647117976, 'first_layer': 512}

batch_size=256
X_drug, y_drug = drug_data, drug_data
train_set_drug  = dataset(X_drug, y_drug)
print(X_drug.shape)
train_dl_drug = DataLoader(train_set_drug, batch_size=batch_size, shuffle=True)
xi, yi = next(iter(train_dl_drug))

def fit():
    lr = param_dict["learning_rate"]
    dropout=param_dict["dropout"]
    weight_decay=param_dict["weight_decay"]
    L1 = param_dict["L1"]
    first_layer=param_dict["first_layer"]

    ae = AE([xi.shape[1],first_layer,512,256,512,first_layer], dropout=dropout)
    print(summary(ae.to("cuda"), xi.shape))
    early_stopper = EarlyStopper(patience=30)
    optimizer = torch.optim.Adam(ae.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=0.5, verbose=True, patience=10, min_lr=1e-7)
    loss_fn = torch.nn.MSELoss()
    epoch_number = 0

    EPOCHS = 500
    for epoch in range(EPOCHS):
        # Make sure gradient tracking is on, and do a pass over the data
        ae.train(True)
        ae = ae.to(device=device)

        avg_loss = train_one_epoch(ae, epoch_number, "writer", train_dl_drug, optimizer, loss_fn, L1=L1,device=device)

        # Set the ae to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        # ae.eval()

        # Disable gradient computation and reduce memory consumption.

        if epoch_number%10==9:print('EPOCH {}:'.format(epoch_number + 1),'LOSS train {}'.format(avg_loss))
        scheduler.step(avg_loss)
        if early_stopper.early_stop(avg_loss):             
            break
        epoch_number+=1
    return ae

ae = fit()
ae.eval()
loss_fn = torch.nn.MSELoss()
with torch.no_grad():
    for i, data in enumerate(train_dl_drug):
        ae = ae.to(device)
        inputs, labels = data
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)
        outputs = ae(inputs)
        loss = loss_fn(outputs, labels) 
        print(loss)
with open("output_drug1.txt", mode="w") as f: 
    print(outputs.shape)
    [f.write(
        str(outputs[0][i].to("cpu").numpy().round(3))+"   "+
        str(inputs[0][i].to("cpu").numpy().round(3))+"\n") for i in range(len(outputs[0]))]


(673, 2412)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 256, 512]       1,235,456
              ReLU-2             [-1, 256, 512]               0
           Dropout-3             [-1, 256, 512]               0
            Linear-4             [-1, 256, 512]         262,656
              ReLU-5             [-1, 256, 512]               0
           Dropout-6             [-1, 256, 512]               0
            Linear-7             [-1, 256, 256]         131,328
            Linear-8             [-1, 256, 512]         131,584
              ReLU-9             [-1, 256, 512]               0
          Dropout-10             [-1, 256, 512]               0
           Linear-11             [-1, 256, 512]         262,656
             ReLU-12             [-1, 256, 512]               0
          Dropout-13             [-1, 256, 512]               0
           Linear-14       

In [1]:
import matplotlib.pyplot as plt
inputs_np = inputs.to("cpu").numpy().round(3)
outputs_np = outputs.to("cpu").numpy().round(3)
inputs_0 = inputs_np[0]
outputs_0 = outputs_np[0]
mask = inputs_0<1

plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.scatter(inputs_0[mask][:100], outputs_0[mask][:100])

NameError: name 'inputs' is not defined

In [35]:
class Encoder(torch.nn.Module):
    def __init__(self, h_sizes, dropout=0.2):
        super().__init__()

        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(h_sizes[0], h_sizes[1]),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(h_sizes[1], h_sizes[2]),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(h_sizes[2], h_sizes[3])
        )
    def forward(self, x):
        encoded = self.encoder(x)
        return encoded

In [41]:
path  ="../models/drug_encoder.pt"
torch.save(ae.encoder.state_dict(), path)

encoder = Encoder([xi.shape[1],param_dict["first_layer"],512,256], dropout=0)
encoder.encoder.load_state_dict(torch.load(path))
encoder.eval()

with torch.no_grad():
    for i, data in enumerate(train_dl_drug):
        encoder = encoder.to(device)
        inputs, labels = data
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)
        outputs = encoder(inputs)
with open("output_encoder_drug.txt", mode="w") as f: 
    [f.write(
        str(outputs[0][i].to("cpu").numpy().round(3))+"   "+
        str(inputs[0][i].to("cpu").numpy().round(3))+"\n") for i in range(len(outputs[0]))]
